In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path

In [32]:
def eliminar_outliers(df, columna):
    # Calcular el IQR
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    # Definir los límites superior e inferior
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    
    # Filtrar los datos eliminando los outliers
    df_filtrado = df[(df[columna] >= lim_inf) & (df[columna] <= lim_sup)]
    
    return df_filtrado


In [41]:
ARCHIVO_VIVIENDA = "data/consolidated/consolidado_vivienda.xlsx"
BASE_FECHA = "2025-10-29"

# Distritos representativos
distritos_sel = ["Miraflores"]

# ---------------- CARGAR DATA ----------------
df_1 = pd.read_excel(ARCHIVO_VIVIENDA)

df = df_1

MIN_PRECIO_M2 = 10  # Precio mínimo por m²
MAX_PRECIO_M2 = 100  # Precio máximo por m²

# Filtrar los datos dentro de este rango de precios
df = df_1[(df_1["precio_m2"] >= MIN_PRECIO_M2) & (df_1["precio_m2"] <= MAX_PRECIO_M2)]
df = eliminar_outliers(df, 'precio_m2')

# Normalizar fecha
df["fecha_publicacion"] = pd.to_datetime(df["fecha_publicacion"])#.dt.strftime('%Y-%m-%d')

df = df[df["precio_m2"] > 0]

# Filtrar las fechas dentro del rango deseado
fecha_inicio = pd.to_datetime("2025-10-29")
fecha_fin = pd.to_datetime("2025-11-19")
df = df[(df['fecha_publicacion'] >= fecha_inicio) & (df['fecha_publicacion'] <= fecha_fin)]



# ---------------- PROMEDIO GEOMÉTRICO ----------------
def geom_promedio(x):
    x = x[x > 0]
    return np.exp(np.log(x).mean()) if len(x) else np.nan

# ---------------- AGRUPACIÓN IPC ----------------
agrupado = (
    df.groupby(["fecha_publicacion", "Tipo"], as_index=False)
      .agg(precio_gmean=("precio_m2", geom_promedio))
)

# ---------------- BASE ----------------
base = (
    agrupado[agrupado["fecha_publicacion"] == BASE_FECHA]
    [["Tipo", "precio_gmean"]]
    .rename(columns={"precio_gmean": "precio_base"})
)

# Merge para calcular índice
merged = agrupado.merge(base, on=["Tipo"], how="left")
merged["indice"] = (merged["precio_gmean"] / merged["precio_base"]) * 100
merged["fecha_publicacion"] = pd.to_datetime(merged["fecha_publicacion"])

# ---------------- TABLA FINAL EN FORMATO LONG ----------------
tabla_long = merged[["Tipo", "fecha_publicacion", "indice"]].copy()
tabla_long = tabla_long.rename(columns={
    "Tipo": "CATEGORIA",
    "fecha_publicacion": "FECHA",
    "indice": "VALOR"
})

# Ordenar columnas + ordenar por categoría y fecha
tabla_long = tabla_long.sort_values(["CATEGORIA", "FECHA"])

# ---------------- GUARDAR ----------------
OUTPUT = "data/base100/base_vivienda.xlsx"
tabla_long.to_excel(OUTPUT, index=False)



C:\Users\jhers\AppData\Local\Temp\ipykernel_52732\3041419749.py:20: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [44]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Supongamos que df_1 es el DataFrame con los datos cargados
# Filtrar solo los alquileres y precios razonables por m²
MIN_PRECIO_M2 = 10  # Precio mínimo por m² (ajustar según el contexto)
MAX_PRECIO_M2 = 100  # Precio máximo por m² (ajustar según el contexto)

# Filtrar solo los alquileres y precios razonables por m²
df_filtrado = df_1[(df_1["precio_m2"] >= MIN_PRECIO_M2) & (df_1["precio_m2"] <= MAX_PRECIO_M2)]

# Eliminar outliers en la columna 'precio_m2'
df_filtrado_sin_outliers = eliminar_outliers(df_filtrado, 'precio_m2')

# Crear el gráfico de cajas para los precios por m2 por distrito usando Plotly
fig = go.Figure()

# Agregar un gráfico de cajas para cada distrito
for distrito in df_filtrado_sin_outliers['distrito'].unique():
    # Filtrar los datos por distrito
    precios_distrito = df_filtrado_sin_outliers[df_filtrado_sin_outliers['distrito'] == distrito]['precio_m2']
    
    # Añadir la traza de cajas para este distrito
    fig.add_trace(go.Box(
        y=precios_distrito,
        name=distrito,
        boxmean='sd',  # Mostrar la media y la desviación estándar
        marker=dict(line=dict(color='black')),  # Color de la línea del gráfico,  # Color de la línea del gráfico
        showlegend=False  # Eliminar la leyenda
    ))

# Configurar el diseño del gráfico
fig.update_layout(
    title=None,#"Distribución de Precios por m² por Distrito (sin Outliers)",
    xaxis_title="Distrito",
    yaxis_title="Precio por m²",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

# Mostrar el gráfico
fig.show()

# Guardar el gráfico como un archivo PDF
fig.write_image("results/Vivienda/boxplot_precios_m2_distrito.pdf")


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Supongamos que df_1 es el DataFrame con los datos cargados
# Filtrar solo los alquileres y precios razonables por m²
MIN_PRECIO_M2 = 10  # Precio mínimo por m² (ajustar según el contexto)
MAX_PRECIO_M2 = 100  # Precio máximo por m² (ajustar según el contexto)

# Filtrar solo los alquileres y precios razonables por m²
df_filtrado = df_1[(df_1["precio_m2"] >= MIN_PRECIO_M2) & (df_1["precio_m2"] <= MAX_PRECIO_M2)]

# Función para eliminar outliers usando el IQR
def eliminar_outliers(df, columna):
    # Calcular el IQR
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    # Definir los límites superior e inferior
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    
    # Filtrar los datos eliminando los outliers
    df_filtrado = df[(df[columna] >= lim_inf) & (df[columna] <= lim_sup)]
    
    return df_filtrado

# Eliminar outliers en la columna 'precio_m2'
df_filtrado_sin_outliers = eliminar_outliers(df_filtrado, 'precio_m2')

# Mapeo de los distritos a valores numéricos para la escala de colores
distritos_unicos = df_filtrado_sin_outliers['distrito'].unique()
distrito_mapping = {distrito: idx for idx, distrito in enumerate(distritos_unicos)}

# Agregar los valores mapeados de los distritos a la columna 'distrito_mapped'
df_filtrado_sin_outliers['distrito_mapped'] = df_filtrado_sin_outliers['distrito'].map(distrito_mapping)

# Crear el gráfico de dispersión para ver la relación entre área y precio por m²
fig2 = go.Figure()

# Agregar gráfico de dispersión (scatter plot) con colores basados en los distritos
fig2.add_trace(go.Scatter(
    x=df_filtrado_sin_outliers['area_m2'],
    y=df_filtrado_sin_outliers['precio_m2'],
    mode='markers',
    marker=dict(
        color=df_filtrado_sin_outliers['distrito_mapped'],  # Usamos el valor numérico de cada distrito
        colorscale='Viridis',  # Escala de colores
        size=10,
        colorbar=dict(title="Distritos")  # Agregar barra de colores
    ),
    text=df_filtrado_sin_outliers['distrito'],  # Información adicional que aparece al pasar el ratón
    name='Área vs Precio por m²'
))

# Configuración del gráfico
fig2.update_layout(
    title='Relación entre Área y Precio por m²',
    xaxis_title='Área (m²)',
    yaxis_title='Precio por m²',
    template='plotly_white',
    width=900,
    height=500
)

fig2.show()

fig.write_image("results/Vivienda/relacion_area_precio.pdf")

C:\Users\jhers\AppData\Local\Temp\ipykernel_52732\1858951124.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
# Generar estadísticas descriptivas de las columnas relevantes
estadisticas = df_filtrado_sin_outliers[['precio_m2', 'area_m2', 'dormitorios', 'banos']].describe()

# Imprimir estadísticas descriptivas
print("Estadísticas descriptivas para las columnas seleccionadas (sin outliers):")
print(estadisticas)

# También puedes acceder a estadísticas específicas, como la media, mediana, etc.
# Ejemplo de cómo acceder a la media de cada columna:
print("\nMedia de cada columna:")
print(df_filtrado_sin_outliers[['precio_m2', 'area_m2', 'dormitorios', 'banos']].mean())

# Ejemplo de cómo acceder a la mediana de cada columna:
print("\nMediana de cada columna:")
print(df_filtrado_sin_outliers[['precio_m2', 'area_m2', 'dormitorios', 'banos']].median())

Estadísticas descriptivas para las columnas seleccionadas (sin outliers):
        precio_m2     area_m2  dormitorios  banos
count  861.000000  861.000000          0.0    0.0
mean    37.069957   96.667828          NaN    NaN
std     10.998366   65.253254          NaN    NaN
min     10.000000   23.000000          NaN    NaN
25%     28.800000   59.000000          NaN    NaN
50%     36.130952   75.000000          NaN    NaN
75%     44.262295  110.000000          NaN    NaN
max     67.800000  450.000000          NaN    NaN

Media de cada columna:
precio_m2      37.069957
area_m2        96.667828
dormitorios          NaN
banos                NaN
dtype: float64

Mediana de cada columna:
precio_m2      36.130952
area_m2        75.000000
dormitorios          NaN
banos                NaN
dtype: float64


In [48]:
# 1. **Estadísticas descriptivas de precios por m² por distrito**
print("Estadísticas descriptivas de 'precio_m2' por distrito (sin outliers):")
estadisticas_precio_distrito = df_filtrado_sin_outliers.groupby('distrito')['precio_m2'].describe()
print(estadisticas_precio_distrito)

# 2. **Estadísticas descriptivas de área por distrito**
print("\nEstadísticas descriptivas de 'area_m2' por distrito (sin outliers):")
estadisticas_area_distrito = df_filtrado_sin_outliers.groupby('distrito')['area_m2'].describe()
print(estadisticas_area_distrito)

# 3. **Estadísticas descriptivas de 'dormitorios' y 'banos'**
print("\nEstadísticas descriptivas de 'dormitorios' y 'banos':")
estadisticas_dormitorios_banos = df_filtrado_sin_outliers[['dormitorios', 'banos']].describe()
print(estadisticas_dormitorios_banos)

# 4. **Relación entre área y precio por m²**
print("\nEstadísticas descriptivas de la relación entre 'area_m2' y 'precio_m2':")
estadisticas_area_precio = df_filtrado_sin_outliers[['area_m2', 'precio_m2']].describe()
print(estadisticas_area_precio)

Estadísticas descriptivas de 'precio_m2' por distrito (sin outliers):
              count       mean        std        min        25%        50%  \
distrito                                                                     
Barranco       72.0  45.682103  10.876532  18.307692  37.656866  44.555556   
Jesús María    46.0  38.131583   7.160394  24.307692  33.333333  38.487526   
La Molina      15.0  28.647152  12.005391  17.777778  21.330526  26.000000   
La Victoria    30.0  41.749978   7.797229  23.684211  37.263889  42.105263   
Lima           50.0  28.701055  11.904095  10.000000  20.738760  26.312217   
Lince          38.0  37.246693   8.684455  20.547945  29.913793  38.585608   
Magdalena      36.0  34.012802  10.091069  19.157088  27.619048  32.160494   
Miraflores    224.0  38.648564   9.374061  21.000000  32.083333  37.347197   
Pueblo Libre   21.0  31.191682   6.185305  23.333333  27.586207  29.850746   
San Borja      31.0  30.704015   6.828500  21.428571  24.905821  30.0000